In [1]:
#import library yang diperlukan
from docx.api import Document
import pandas as pd
import re
import numpy as np
from bs4 import BeautifulSoup
import nltk
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemover import StopWordRemover
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Dictionary.ArrayDictionary import ArrayDictionary
from nltk.tokenize import WordPunctTokenizer

In [3]:
#convert file docx menjadi csv
document = Document('DataPengenalanPola.docx')
df = pd.DataFrame()

for i in range(len(document.tables)):
    table = document.tables[i]
    keys = None
    data = []
    
    for i, row in enumerate(table.rows):
        text = (cell.text for cell in row.cells)
        if i == 0:
            keys = tuple(text)
            continue
        row_data = dict(zip(keys, text))
        data.append(row_data)
        #print(data)
    data2 = pd.DataFrame(data)
    df = df.append(data2,verify_integrity=True,sort=True,ignore_index=True)

In [4]:
df = df.drop('\n\nNo',axis=1)
df = df.drop([0],axis=0)

In [5]:
pat1 = r'\n'
df.iloc[:,0] = [re.sub(pat1,' ',str(df.iloc[i,0])) for i in range(len(df))]
df.iloc[:,1] = [re.sub(pat1,' ',str(df.iloc[i,1])) for i in range(len(df))]

In [7]:
#menyimpan ke dalam file csv
df.to_csv('DataPengenalanPola_clean.csv', encoding='utf-8')

In [8]:
df

,Tanda dan Gejala (Query),Nama Penyakit (Data Diagnosis)
1,"Menggigil, demam, sakit kepala",Malaria (bentuk benigma)
2,"Kaku kuduk, penurunan kesadaran, muntah proyek...",Meningitis + perdarahan subarachnoid
3,"Mata lengket, mata berair, pandangan sedikit k...",
4,"Pipi bengkak, nyeri saat mengunyah, nyeri testis",Parotitis
5,"Gusi bengkak, gusi kemerahan, gusi berdarah",
6,"Sakit gigi, gigi sensitif pada makanan dingin ...",Karies dentis
7,"Hidung tersumbat, bersin, batuk, sakit tenggor...",Common cold
8,"Hidung keluar lendir, choncae nasal kebiruan, ...",Rinitis alergi
9,"Hidung berlendir, demam, choncae nasal kemerah...",Rinitis bacterial
10,"Hidung bau, nyeri di dalam hidung, ada krusta ...",Rinitis sika


In [193]:
#load dataset
data = pd.read_csv('DataPengenalanPola_clean.csv', names=['No','Gejala','Diagnosis'],header=0)

In [194]:
data.head()

,No,Gejala,Diagnosis
0,1,"Menggigil, demam, sakit kepala",Malaria (bentuk benigma)
1,2,"Kaku kuduk, penurunan kesadaran, muntah proyek...",Meningitis + perdarahan subarachnoid
2,3,"Mata lengket, mata berair, pandangan sedikit k...",
3,4,"Pipi bengkak, nyeri saat mengunyah, nyeri testis",Parotitis
4,5,"Gusi bengkak, gusi kemerahan, gusi berdarah",NaN


In [195]:
data.shape

(100, 3)

In [196]:
index = [2,18,19,29,36,76,4]
test = data.iloc[[2,18,19,29,36,76,4],:]
data = data.drop([2,18,19,29,36,76,4],axis=0)
data = data.reset_index(drop=True)
test = test.reset_index(drop=True)

In [159]:
#preprocessing

stemfactory = StemmerFactory()
stemmer = stemfactory.create_stemmer()
tok = WordPunctTokenizer()
factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()

def cleaner(text):
    #hanya mengambil huruf pada text
    letters_only = re.sub("[^a-zA-Z]"," ", text)
    
    #mengubah text menjadi lower case
    lower_case = letters_only.lower()
    
    #proses stemming
    stem_words = stemmer.stem(lower_case)
    
    #stopwords removal
    stopword.remove(stem_words)
    
    #tokenisasi
    words = tok.tokenize(stem_words)
    
    return(words)

In [197]:
#melakukan cleaning terhadap dataset dan menyimpannya ke dalam list gejala
gejala = []
for i in range(len(data)):
    gejala.append(cleaner(data.iloc[i,1]))

In [199]:
#mengambil semua term unik dari list gejala
terms = []
for i in range(93):
    for w in range(len(gejala[i])):
        if gejala[i][w] not in terms:
            terms.append(gejala[i][w])
len(terms)

394

In [200]:
#menambahkan kolom terms pada dataframe data dan test
for i in range(len(terms)):
    data.insert(3, str(terms[i]),0,allow_duplicates=False)
    test.insert(3,str(terms[i]),0,allow_duplicates=False)

In [201]:
data.shape

(93, 397)

In [202]:
test.shape

(7, 397)

In [203]:
#menambahkan term frequency pada setiap kolom term berdasarkan gejala
for col in reversed(range(394)):
    for row in range(93):
        data.loc[row, terms[col]] = gejala[row].count(terms[col])

In [204]:
data

,No,Gejala,Diagnosis,spora,hifa,khas,tanda,dengung,eksudatif,sangat,...,proyektil,muntah,sadar,turun,kuduk,kaku,kepala,sakit,demam,gigil
0,1,"Menggigil, demam, sakit kepala",Malaria (bentuk benigma),0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,1,1
1,2,"Kaku kuduk, penurunan kesadaran, muntah proyek...",Meningitis + perdarahan subarachnoid,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,0,0
2,4,"Pipi bengkak, nyeri saat mengunyah, nyeri testis",Parotitis,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,"Sakit gigi, gigi sensitif pada makanan dingin ...",Karies dentis,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,7,"Hidung tersumbat, bersin, batuk, sakit tenggor...",Common cold,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5,8,"Hidung keluar lendir, choncae nasal kebiruan, ...",Rinitis alergi,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,9,"Hidung berlendir, demam, choncae nasal kemerah...",Rinitis bacterial,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
7,10,"Hidung bau, nyeri di dalam hidung, ada krusta ...",Rinitis sika,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,11,"Cuping telinga sakit, cuping telinga berdungku...",Perikondritis,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
9,12,"Telinga sakit, keluar lendir dari telinga, pen...",Otitis eksterna,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [205]:
#melakukan cleaning pada data test
test_gejala = []
for i in range(len(test)):
    test_gejala.append(cleaner(test.iloc[i,1]))

In [207]:
#mengisi term frequency pada setiap kolom term sesuai dengan gejala
for col in reversed(range(394)):
    for row in range(len(test)):
        test.loc[row, terms[col]] = test_gejala[row].count(terms[col])

In [208]:
test

,No,Gejala,Diagnosis,spora,hifa,khas,tanda,dengung,eksudatif,sangat,...,proyektil,muntah,sadar,turun,kuduk,kaku,kepala,sakit,demam,gigil
0,3,"Mata lengket, mata berair, pandangan sedikit k...",,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,19,"Batuk lebih dari tiga minggu, sesak napas atau...",,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,20,"Demam, menggigil, suhu tubuh meningkat, batuk ...",NaN,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
3,30,"Demam, muntah, diare cair, ampas sedikit seper...",,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,37,"Nyeri kolik daerah pinggang, malaise, mual, ke...",,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,77,Ruam yang gatal terdri dari macula-makula keci...,,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
6,5,"Gusi bengkak, gusi kemerahan, gusi berdarah",NaN,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
#TF-IDF
def frekuensi(word, doc):
    return doc.count(word)

def tf(word, doc):
    return (frekuensi(word,doc)/float(len(doc)))

def num_docs_containing(word, list_of_docs):
    count = 0
    for document in list_of_docs:
        if frekuensi(word,document)>0:
            count+=1
    return 1+count
def idf(word, list_of_docs):
    return np.log(len(list_of_docs)/
                 float(num_docs_containing(word,list_of_docs)))
def tf_idf(word, doc, list_of_docs):
    return (tf(word,doc)*idf(word,list_of_docs))

In [209]:
data2 = data.copy()

In [210]:
#menghitung TF-IDF
for col in reversed(range(394)):
    for row in range(93):
        data2.loc[row, terms[col]] = tf_idf(terms[col], gejala[row], gejala) 

In [211]:
data2

,No,Gejala,Diagnosis,spora,hifa,khas,tanda,dengung,eksudatif,sangat,...,proyektil,muntah,sadar,turun,kuduk,kaku,kepala,sakit,demam,gigil
0,1,"Menggigil, demam, sakit kepala",Malaria (bentuk benigma),0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.533676,0.360389,0.210930,0.613289
1,2,"Kaku kuduk, penurunan kesadaran, muntah proyek...",Meningitis + perdarahan subarachnoid,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.479932,0.278752,0.479932,0.342605,0.429248,0.365395,0.266838,0.180195,0.000000,0.000000
2,4,"Pipi bengkak, nyeri saat mengunyah, nyeri testis",Parotitis,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,6,"Sakit gigi, gigi sensitif pada makanan dingin ...",Karies dentis,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.131051,0.000000,0.000000
4,7,"Hidung tersumbat, bersin, batuk, sakit tenggor...",Common cold,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.240260,0.000000,0.000000
5,8,"Hidung keluar lendir, choncae nasal kebiruan, ...",Rinitis alergi,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,9,"Hidung berlendir, demam, choncae nasal kemerah...",Rinitis bacterial,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.105465,0.000000
7,10,"Hidung bau, nyeri di dalam hidung, ada krusta ...",Rinitis sika,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,11,"Cuping telinga sakit, cuping telinga berdungku...",Perikondritis,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.180195,0.105465,0.000000
9,12,"Telinga sakit, keluar lendir dari telinga, pen...",Otitis eksterna,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.131051,0.000000,0.000000


In [213]:
#menghitung TF-IDF untuk data test
for col in reversed(range(394)):
    for row in range(len(test)):
        test.loc[row, terms[col]] = tf_idf(terms[col], test_gejala[row], test_gejala) 

In [214]:
test

,No,Gejala,Diagnosis,spora,hifa,khas,tanda,dengung,eksudatif,sangat,...,proyektil,muntah,sadar,turun,kuduk,kaku,kepala,sakit,demam,gigil
0,3,"Mata lengket, mata berair, pandangan sedikit k...",,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000
1,19,"Batuk lebih dari tiga minggu, sesak napas atau...",,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.062638,0.0,0.0,0.0,0.0,0.000000,0.000000
2,20,"Demam, menggigil, suhu tubuh meningkat, batuk ...",NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.039973,0.089483
3,30,"Demam, muntah, diare cair, ampas sedikit seper...",,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.113888,0.0,0.000000,0.0,0.0,0.0,0.0,0.050874,0.000000
4,37,"Nyeri kolik daerah pinggang, malaise, mual, ke...",,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000
5,77,Ruam yang gatal terdri dari macula-makula keci...,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.029453,0.000000
6,5,"Gusi bengkak, gusi kemerahan, gusi berdarah",NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000


In [226]:
#fungsi untuk menghitung cosine similarity
def cosine(doc1,doc2):
    cs = np.dot(doc1,doc2.T)/np.dot((np.sqrt(np.sum(np.power(doc1,2)))),(np.sqrt(np.sum(np.power(doc2,2)))).T)
    return cs

In [283]:
#menghitung similarity untuk setiap data test
similarity = []
for i in range(len(test)):
    compute_cosine = []
    for j in range(len(data2)):
        compute_cosine.append(cosine(test.iloc[i,3:],data2.iloc[j,3:]))
    similarity.append(compute_cosine.index(max(compute_cosine)))

In [234]:
#hasil cosine similarity
similarity

[89, 67, 61, 24, 41, 78, 3]

In [274]:
test['Diagnosis'] = data2.iloc[[89, 67, 61, 24, 41, 78, 3],2].values

In [280]:
test.iloc[:,1:3]

,Gejala,Diagnosis
0,"Mata lengket, mata berair, pandangan sedikit k...",Conjunctivitis
1,"Batuk lebih dari tiga minggu, sesak napas atau...",Tuberkulosis
2,"Demam, menggigil, suhu tubuh meningkat, batuk ...",Pneumonia
3,"Demam, muntah, diare cair, ampas sedikit seper...",Kolera
4,"Nyeri kolik daerah pinggang, malaise, mual, ke...",Pelvic inflammatory disease
5,Ruam yang gatal terdri dari macula-makula keci...,Rubela (campak jerman)
6,"Gusi bengkak, gusi kemerahan, gusi berdarah",Karies dentis
